In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import copy


In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_WORKERS = 0
NUM_EPOCHS = 50

In [ ]:
#TODO
dataset = {
    'train':
        Dataset(X_train, y_train),
    'valid':
        Dataset(X_train, y_train)
}

data_loader = {
    'train':
        torch.utils.data.DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS),
    'valid':
        torch.utils.data.DataLoader(dataset['valid'], batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
}

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16 * 543, 250)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.flatten(x)
        x = self.fc5(x)
        return self.softmax(x)
model = Net()


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=5e-5)

In [ ]:
train_losses = []
valid_losses = []
train_acc = []
valid_acc = []
lr = [0.0001]
def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(id_2_sign_map[preds] == id_2_sign_map[labels]) # from Dataset (test)

            lr_scheduler.step()
            lr.append(optimizer.param_groups[0]["lr"])
            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects.double() / len(dataset[phase])
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_losses.append(epoch_loss)
                valid_acc.append(epoch_acc)
                if best_acc < epoch_acc: 
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model.load_state_dict(best_model_wts)

In [12]:
def legend(list_1, label_1, step, title, list_2=None, label_2=None):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(list_1[1:], label=label_1)
    if list_2 is not None:
        plt.plot(list_2[1:], label=label_2)
    plt.xticks(range(1, NUM_EPOCHS + step, step))
    plt.legend()
    plt.xlabel('Iterations')
    plt.ylabel(title)
    plt.show()
legend(list_1 = train_losses, label_1='Train', step = 5, title='Loss', list_2 = valid_losses, label_2='Valid')
legend(list_1 = train_acc, label_1='Train', step = 5, title='Accuracy', list_2 = valid_acc, label_2='Valid')
legend(list_1 = lr, label_1='Lr', step = 5, title='Learning rate')


NameError: name 'train_losses' is not defined